# Calculation of Mortality Rate – Demographic Flow Method  
*(Argentina Example – Reverse Engineering Mortality from Population Balances)*

## 1. Core Idea: Population as a Markov-like Flow System

Every year, the total population changes due to four main flows:

$$
P_{t+1} = P_t + \text{Births}_t + \text{Immigration}_t - \text{Emigration}_t - \text{Deaths}_t
$$

Where:  
- $P_t$ = Total population at the end of year $t$ (or beginning of $t+1$)  
- $\text{Births}_t$ = Live births registered during year $t$  
- $\text{Deaths}_t$ = Total deaths during year $t$ → **this is what we want to estimate**  
- $\text{Immigration}_t$ and $\text{Emigration}_t$ = International migration flows  

This is the classic **demographic balancing equation**.

## 2. Solving for Mortality (Deaths)

If we have reliable data for $P_{t+1}$, $P_t$, births, and net migration (or separate immigration and emigration), we can rearrange the equation to isolate deaths:

$$
\boxed{
\text{Deaths}_{t} = P_t + \text{Births}_{t} + \text{Immigration}_{t} - \text{Emigration}_{t} - P_{t+1}
}
$$

Or, using **net migration** (Immigration − Emigration):

$$
\boxed{
\text{Deaths}_{t} = P_t + \text{Births}_{t} + \text{Net Migration}_{t} - P_{t+1}
}
$$

### Example: Estimating Deaths in 2023 (Argentina)

$$
\text{Deaths}_{2023} = P_{2022} + \text{Births}_{2023} + \text{Immigration}_{2023} - \text{Emigration}_{2023} - P_{2023}
$$

This method is especially useful when official death records are incomplete, delayed, or questionable (e.g., during pandemic years), but population estimates, births, and migration data are available.

## 3. Crude Mortality Rate (per 1,000 inhabitants)

Once we have estimated deaths, the crude death rate (CDR) is:

$$
\text{CDR}_{t} \left( \text{per 1,000} \right) = \frac{\text{Deaths}_{t}}{\text{Mid-year Population}_{t}} \times 1000
$$

Usually we approximate mid-year population as:

$$
\text{Mid-year Population}_{t} \approx \frac{P_t + P_{t+1}}{2}
$$

or use the official mid-year estimate when available.

## 4. Why This Method Works Well for Argentina

Argentina publishes:
- Annual population estimates/projections by INDEC (usually December 31 or July 1)
- Definitive birth series (Vital Statistics)
- Migration estimates (often net migration from National Directorate of Migration + censuses)

→ This gives us all the inputs needed to **back-calculate total deaths** with high accuracy, even before official mortality tables are released.